# Movie Bot prototyp

Should help find movies in a conversational way.
Output Movie Recommendation and info howto stream.

## Steps

- [X] Get Movie Recommendations via LLM and userinput
- [X] Use LLM function call to get the titles
- [X] search movies via TMDB Api and find streaming provider **AND AALL OTHER MOVIE_INFO DATA (PLAYTIME; ACTORS; GENRES)**
- [X] write recommendation to chat output with context form 3
- [X] save history
- [X] use system propmpts
- [] ask for wich providers




### Bonus Tasks
- [ ] find newer once via tmdb api and recommendet titles
- [ ] if user asks further questions answer based on api call
- [ ] if user asks for new recros do this! - do not recro same movies in one session
- [ ] build website with chat interface
- [ ] build website with images and direct links
- [ ] filter for providers and pref to rent / flatrate
- [ ] filter this by user input
- [ ] use block/watch list / streaming source and so on


In [1]:
from dotenv import load_dotenv

import os

#global sessionvars
blacklists=[]

# Setup keys
load_dotenv()
openai_api_key = os.getenv('openai_api_key')
tmdb_bearer = os.getenv('tmdb_bearer')


In [2]:
from openai import OpenAI
client = OpenAI(
    api_key=openai_api_key,
)


In [18]:
#tmdb api
# moviedb tool singe title search
import requests
import json
from difflib import get_close_matches
import traceback 
import urllib.parse

# If you're curious of all the loggers
#print(streamlit.logger._loggers)  

#streamlit_root_logger = logging.getLogger(streamlit.__name__)


headers = {
    "accept": "application/json",
    "Authorization":  tmdb_bearer
}




def get_movies_with_recro(titles: list[str], providers: list[str], blacklist: str):
  #print(f"get_basic_data_from_tmdb_for_titles",titles)
  movies =[]
  # get movies from tmdb from given titles
  for title in titles:
    movie = get_basic_data_from_tmdb_for_title(title)
    if movie:
      movies.append(movie)
  # add similar recommendations from tmdb
  similar_movies = get_recro_movies(movies)
  if similar_movies:
    movies = movies + similar_movies
  # filter for providers
  rsult_movies = filter_movies(movies, providers, blacklist)
  if rsult_movies and len(rsult_movies) > 5:
    rsult_movies = rsult_movies[:4]    
  fulldata_as_string = create_data_list(rsult_movies)
  return fulldata_as_string
  

def get_basic_data_from_tmdb_for_titles(titles: list[str]):
  print(f"get_basic_data_from_tmdb_for_titles",titles)
  movies =[]
  for title in titles:
    movie = get_basic_data_from_tmdb_for_title(title)
    if movie:
      movies.append(movie)
  fulldata_as_string = create_data_list(movies)
  return fulldata_as_string


def get_detail_data_from_tmdb_for_titles(titles: list[str]):
  #print(f"get_detail_data_from_tmdb_for_titles",titles)
  movies =[]
  for title in titles:
    movie = get_detail_data_from_tmdb_for_title(title)
    if movie:
      movies.append(movie)
  return movies

def get_basic_data_from_tmdb_for_title(title: str):
  try:
    #print(f"get_movie_data_from_tmdb","title=",title)
    fulldata = create_basic_movie_data(title)
    return fulldata
  except:
     return None

def get_detail_data_from_tmdb_for_title(title: str):
  try:
    #print(f"get_movie_data_from_tmdb","title=",title)
    fulldata = create_movie_data(title)
    return fulldata
  except:
     return None

def get_recro_movies(movies):  
  if len(movies) > 0:
    searchId = movies[0]["id"]
    results = find_smilar_movies(searchId)
    if results:
      movies = parse_movies_from_search(results)
    return movies



def find_person_id(searchquery:str):
  searchstring = urllib.parse(searchquery)
  url = "https://api.themoviedb.org/3/search/person?query="+str(searchstring)+"&include_adult=false&language=de-DE&page=1"  
  response = requests.get(url, headers=headers)  
  if response.status_code == 200:
    data = json.loads(response.text)  
    return data
  return None 

def find_content_by_personid(id:str):    
  url = "https://api.themoviedb.org/3/person/"+str(id)+"/movie_credits?language=de-DE"  
  response = requests.get(url, headers=headers)  
  if response.status_code == 200:
    data = json.loads(response.text)  
    return data
  return None 

def find_smilar_movies(id):
  url = "https://api.themoviedb.org/3/movie/"+str(id)+"/recommendations?language=en-US&page=1"
  response = requests.get(url, headers=headers)  
  if response.status_code == 200:
    data = json.loads(response.text)  
    return data
  return None  

def parse_movies_from_search(results):
  movies=[]
  for movie in results["results"]:
    if movie and "id" in movie:
      id = get_movie_id(movie)
      providers = get_watch_providers(id)
      flatproviders = get_watch_providers_via_subtype(filter_watch_providers(providers, "DE"),"flatrate" )
      rentproviders = get_watch_providers_via_subtype(filter_watch_providers(providers, "DE"),"rent" )
      #buyproviders = get_watch_providers_via_subtype(filter_watch_providers(providers, "DE"),"buy" )

      filtered_data = {'title': movie['title'], 'flatproviders': flatproviders, 'rentproviders': rentproviders, 'overview': movie['overview'],
                      'release_date': movie['release_date'], 'id': id,
                      }
      movies.append(filtered_data)
  return movies

def find_movie_basic(title, lang):

  title = title.replace(' ', '+')
  url = "https://api.themoviedb.org/3/search/movie?include_adult=false&language="+lang+"&page=1&query="+ title

  response = requests.get(url, headers=headers)

  if response.status_code == 200:
    data = json.loads(response.text)
    #print(f"find_movie for", title, "response: ",data)

    if "results" in str(data) and len(data["results"]) > 0:
      #print(f"movie found!")
      movie = get_movie_form_search(data["results"], title)
      return movie

  return None

def find_movie(title, lang):

  title = title.replace(' ', '+')
  url = "https://api.themoviedb.org/3/search/movie?include_adult=false&language="+lang+"&page=1&query="+ title

  response = requests.get(url, headers=headers)

  if response.status_code == 200:
    data = json.loads(response.text)
    #print(f"find_movie for", title, "response: ",data)

    if "results" in str(data) and len(data["results"]) > 0:
      #print(f"movie found!")
      movie = get_movie_form_search(data["results"], title)
      movie = get_detail_moviedata(get_movie_id(movie))
      movie = json.loads(movie)
      return movie

  return None


def get_movie_id(movie):
  #print(f"get_movie_id for ", movie)
  if movie:
      return movie["id"]


def get_watch_providers(id):
  #print(id)
  url = "https://api.themoviedb.org/3/movie/"+str(id)+"/watch/providers"
  response = requests.get(url, headers=headers)
  return response.text

def get_detail_moviedata(id):
  url = "https://api.themoviedb.org/3/movie/"+str(id)+"?language=en-US"
  response = requests.get(url, headers=headers)
  return response.text


def filter_watch_providers(data, lang):
  # Filter data under "<lang>"
  data = json.loads(data)
  if lang in data["results"]:
    filtered_data = data["results"][lang]
    # Print the filtered JSON
    return filtered_data



def get_watch_providers_via_subtype(data, subtype):
#subtype = "bye", "rent", "flatrate"

  provider_names=[]
  if subtype in str(data):
    filtered_data = data[subtype]
    if len(filtered_data) > 0:
      provider_names = [item['provider_name'] for item in filtered_data]

  return provider_names


def create_movie_data(title):
  movie = find_movie(title,"de-DE")
  if movie and "id" in movie:
    id = get_movie_id(movie)
    providers = get_watch_providers(id)
    flatproviders = get_watch_providers_via_subtype(filter_watch_providers(providers, "DE"),"flatrate" )
    rentproviders = get_watch_providers_via_subtype(filter_watch_providers(providers, "DE"),"rent" )
    buyproviders = get_watch_providers_via_subtype(filter_watch_providers(providers, "DE"),"buy" )
    genres = [x['name'] for x in movie['genres']]
    prodcountries = [x['name'] for x in movie['production_countries']]
    filtered_data = {'title': movie['title'], 'flatproviders': flatproviders, 'rentproviders': rentproviders, 'buyproviders': buyproviders,
                     'adult': movie['adult'], 'genres': genres, 'budget': movie['budget'], 'overview': movie['overview'],
                     'popularity': movie['popularity'], 'poster_path':movie['poster_path'], 'release_date': movie['release_date'],
                     'runtime': movie['runtime'], 'revenue': movie['revenue'], 'vote_average':movie['vote_average'], 'vote_count': movie['vote_count'],
                      'production_countries': prodcountries
                     }
    return filtered_data

def create_basic_movie_data(title):
  movie = find_movie_basic(title,"de-DE")
  if movie and "id" in movie:
    id = get_movie_id(movie)
    providers = get_watch_providers(id)
    flatproviders = get_watch_providers_via_subtype(filter_watch_providers(providers, "DE"),"flatrate" )
    rentproviders = get_watch_providers_via_subtype(filter_watch_providers(providers, "DE"),"rent" )
    buyproviders = get_watch_providers_via_subtype(filter_watch_providers(providers, "DE"),"buy" )

    filtered_data = {'title': movie['title'], 'flatproviders': flatproviders, 'rentproviders': rentproviders, 'overview': movie['overview'],
                     'release_date': movie['release_date'], 'id': id,
                     }
    return filtered_data




def create_recommendation_data(item):

  ## just importent info
  filtered_data = {'title': item['title'], 'flatproviders': item['flatproviders'], 'rentproviders': item['rentproviders'], 'buyproviders': item['buyproviders']}
  ##filter only movies that are for streaming or rent available
  item =filtered_data

  if 'flatproviders' in item and len(item['flatproviders']) > 0:
      if 'rentproviders' in item and len(item['rentproviders']) > 0:
           return {"title":item['title'], 'flatproviders': item['flatproviders'], 'rentproviders': item['rentproviders']}
      else:
          return {"title":item['title'],'flatproviders': item['flatproviders']}
  elif 'rentproviders' in item and len(item['rentproviders']) > 0:
     return {"title":item['title'],'rentproviders': item['rentproviders']}

def get_movie_form_search(data:dict, search:str):
  try:
      titles = [x["title"] for x in data]
      title = closeMatches(titles, search)
      title = title[0]
      movie = [x for x in data if x['title'] == title]
      return movie[0]
  except:
    return None

# Function to find all close matches of
# input string in given list of possible strings
def closeMatches(patterns, word):
     return get_close_matches(word, patterns)

def create_data_list(fulldata):
  movieinfoasstring=""
  for movie_info in fulldata:
     formatted_info = f"Title: {movie_info['title']}, "
     formatted_info += f"Overview: {movie_info['overview']}, "
     formatted_info += f"flatproviders: {', '.join(movie_info['flatproviders'])}, "
     formatted_info += f"rentproviders: {', '.join(movie_info['rentproviders'])}\n"
     movieinfoasstring+=formatted_info
  return movieinfoasstring

def filter_movies(movies: dict, providers: list[str], blacklist):  
  filtered_list = movies
  if providers and len(providers) > 0:  
    filtered_list = [d for d in movies if ('flatproviders' in d and any(val in d['flatproviders'] for val in providers)) or ('rentproviders' in d and any(val in d['rentproviders'] for val in providers))]  
    for movie in filtered_list:
      flatproviders=[]
      rentproviders=[]
      for provider in providers:
        if provider in movie["flatproviders"]:
          flatproviders.append(provider)
        if provider in movie["rentproviders"]:
          rentproviders.append(provider)  
      movie["rentproviders"]=rentproviders
      movie["flatproviders"]=flatproviders
  
  filtered_list = [m for m in filtered_list if len(m["flatproviders"])>0 or len(m["rentproviders"])>0]
  filtered_list = [m for m in filtered_list if m["title"] not in blacklist]

  return filtered_list


#helpers
def get_movie_data_by_titlelist(titles: list):
  try:
#    titles = input["titles"]
    print(f"_get_movie_data titles input: ", titles)
    #inputlist = titles.split(",")
    #inputlist = [x.strip(' ') for x in inputlist]      
    result = get_basic_data_from_tmdb_for_titles(titles)
    print(f"get_movie_data result: ", result)
    return result
  except:    
     traceback.print_exc() 
     return None


def get_movie_data(input: dict):
  try:
    titles = input["titles"]
    print(f"_get_movie_data titles input: ", titles)
    inputlist = titles.split(",")
    inputlist = [x.strip(' ') for x in inputlist]      
    result = get_basic_data_from_tmdb_for_titles(inputlist)
    print(f"get_movie_data result: ", result)
    return result
  except:    
     traceback.print_exc() 
     return None

def get_recro_movies_old(input: dict):
  try:
    #titles = input["titles"]
    titles = list(input["titles"]["text"].values())
    inputlist = titles
    provider = input["config"]["provider"]
    blacklist = input["config"]["blacklist"]
    print(f"_get_movie_data titles input: ", titles)
    #inputlist = titles.split(",")
    #inputlist = [x.strip(' ') for x in inputlist]
    result = get_movies_with_recro(inputlist, provider, blacklist)
    print(f"get_recro_movies result: ",result)
    return result  
  except:
      traceback.print_exc() 
      return None

def get_movie_recommendations_by_title(titles, STREAMING_PROVIDER = ["Disney Plus","Amazon Prime Video","Apple TV","MagentaTV", "Netflix"] ):
  try:
    #titles = input["titles"]
    #titles = list(input["titles"]["text"].values())
    inputlist = titles
    #provider = input["config"]["provider"]
    #blacklist = input["config"]["blacklist"]
    print(f"_get_movie_data titles input: ", titles)
    #inputlist = titles.split(",")
    #inputlist = [x.strip(' ') for x in inputlist]
    provider = STREAMING_PROVIDER 
    blacklist = []
    result = get_movies_with_recro(inputlist, provider, blacklist)
    print(f"get_recro_movies result: ",result)
    return result  
  except:
      traceback.print_exc() 
      return None

def get_detail_moviedata(title: str):
  try:
    print(f"get_movie_data_from_tmdb","title=",title)
    fulldata = get_detail_data_from_tmdb_for_title(title)
    print(f"get_detail_moviedata","result:", fulldata)
    return fulldata
  except:
     traceback.print_exc() 
     return None
  


# find movie by actor
# find movies by regisseur

# add newest ones via similarity search
  



In [ ]:
# test tmdb helper

testtitlelist = {    
        "text": {
            "1": "Transformers",
            "2": "RoboCop",
            "3": "Robot",
            "4": "Bladerunner",
        }    
}

resp = get_recro_movies({"titles": testtitlelist,"config":{"provider": ["Disney Plus","MagentaTV"], "blacklist": [] }})  

In [19]:
testtitlelist =  "Transformers, RoboCop, Robot, Blade Runner"
#get_movie_data({"titles":testtitlelist})

get_movie_recommendations_by_title(titles=testtitlelist)

_get_movie_data titles input:  Transformers, RoboCop, Robot, Blade Runner
get_recro_movies result:  


''

In [20]:
titles = ['The Blues Brothers', 'Tenacious D in The Pick of Destiny', 'Pineapple Express']

get_movie_recommendations_by_title(titles)

_get_movie_data titles input:  ['The Blues Brothers', 'Tenacious D in The Pick of Destiny', 'Pineapple Express']
get_recro_movies result:  Title: Blues Brothers, Overview: Gleich nachdem Jake Blues aus dem Gefängnis entlassen wird, besuchen er und sein Bruder Elwood das Waisenhaus, in dem sie großgezogen wurden. Dort müssen sie erfahren, dass das Waisenhaus an das Kultusministerium verkauft werden soll. Es gibt nur eine Chance auf Rettung: Innerhalb von elf Tagen muss die Grundsteuer bezahlt werden. Die Brüder beschließen das Geld aufzutreiben, indem sie ihre alte Blues-Band wieder zusammenstellen und ein großes Konzert geben., flatproviders: , rentproviders: Apple TV, MagentaTV
Title: Blues Brothers 2000, Overview: Finally released from prison, Elwood Blues is once again enlisted by Sister Mary Stigmata in her latest crusade to raise funds for a children's hospital. Hitting the road to re-unite the band and win the big prize at the New Orleans Battle of the Bands, Elwood is pursued cr

"Title: Blues Brothers, Overview: Gleich nachdem Jake Blues aus dem Gefängnis entlassen wird, besuchen er und sein Bruder Elwood das Waisenhaus, in dem sie großgezogen wurden. Dort müssen sie erfahren, dass das Waisenhaus an das Kultusministerium verkauft werden soll. Es gibt nur eine Chance auf Rettung: Innerhalb von elf Tagen muss die Grundsteuer bezahlt werden. Die Brüder beschließen das Geld aufzutreiben, indem sie ihre alte Blues-Band wieder zusammenstellen und ein großes Konzert geben., flatproviders: , rentproviders: Apple TV, MagentaTV\nTitle: Blues Brothers 2000, Overview: Finally released from prison, Elwood Blues is once again enlisted by Sister Mary Stigmata in her latest crusade to raise funds for a children's hospital. Hitting the road to re-unite the band and win the big prize at the New Orleans Battle of the Bands, Elwood is pursued cross-country by the cops., flatproviders: , rentproviders: Apple TV, MagentaTV\nTitle: Animal House, Overview: At a 1962 College, Dean Ver

In [21]:
from openai import OpenAI
import json



def run_conversation():
    # Step 1: send the conversation and available functions to the model
    messages = [{"role": "user", "content": "Empfehle mir bitte spannede und lustige Filme aus den 80ern und sage mir wo ich sie sehen kann!"}]
    tools = [
        {
            "type": "function",
            "function": {
                "name": "get_movie_data",
                "description": "get all needed information about a list of movietitles regarding streeming providers",
                "parameters": {
                    'type': 'object',
                    'properties': {
                        'titles': {
                            'type': 'array',
                            'items': {
                                'type': 'string'                         
                            },
                            'description': 'list of movie titles from given input text'
                        }                        
                    },
                    "required": ["titles"],
                }
            }
        }
    ]
    response = client.chat.completions.create(
        model="gpt-3.5-turbo-0125",
        messages=messages,
        tools=tools,
        tool_choice="auto",  # auto is default, but we'll be explicit
    )
    response_message = response.choices[0].message
    tool_calls = response_message.tool_calls
    # Step 2: check if the model wanted to call a function
    if tool_calls:
        # Step 3: call the function
        # Note: the JSON response may not always be valid; be sure to handle errors
        available_functions = {
            "get_movie_data": get_movie_recommendations_by_title,
        }  # only one function in this example, but you can have multiple
        messages.append(response_message)  # extend conversation with assistant's reply
        # Step 4: send the info for each function call and function response to the model
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)
            function_response = function_to_call(
                titles=function_args.get("titles")                
            )
            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response,
                }
            )  # extend conversation with function response
        second_response = client.chat.completions.create(
            model="gpt-3.5-turbo-0125",
            messages=messages,
        )  # get a new response from the model where it can see the function response
        return second_response
print(run_conversation())

_get_movie_data titles input:  ['Back to the Future', 'Ghostbusters', 'Die Goonies', 'Beverly Hills Cop', 'Ferris macht blau', 'Zurück in die Zukunft', 'Der Prinz aus Zamunda', 'Big', 'Blues Brothers', 'Stirb langsam', 'Die nackte Kanone', 'Indiana Jones und der Tempel des Todes', 'Karate Kid', 'Rambo: First Blood Part II', 'Zurück in die Zukunft 2']
get_recro_movies result:  Title: Ghostbusters, Overview: Abby Yates hat schon immer an Gespenster geglaubt, obwohl sie dafür belächelt wird. In einem Labor erforscht sie das Paranormale, unterstützt von der schrägen, aber genialen Ingenieurin Jillian Holtzmann, die abgefahrene Anti-Geister-Waffen bastelt. Abbys alte Freundin Erin Gilbert hat das umstrittene Forschungsgebiet hingegen für eine Karriere als seriöse Physikerin aufgegeben, was einst zum Bruch zwischen den beiden Partnerinnen führte. Doch als Erin kurz vor einer Festanstellung an der renommierten Columbia-Universität steht, wird sie von der gemeinsamen Geisterforschervergangenhe

In [40]:
from openai import OpenAI
import json



def run_conversation(messages):
    # Step 1: send the conversation and available functions to the model   
    tools = [
        {
            "type": "function",
            "function": {
                "name": "get_movie_data",
                "description": "get all needed information about a list of movietitles regarding streeming providers",
                "parameters": {
                    'type': 'object',
                    'properties': {
                        'titles': {
                            'type': 'array',
                            'items': {
                                'type': 'string'                         
                            },
                            'description': 'list of movie titles from given input text'
                        }                        
                    },
                    "required": ["titles"],
                }
            }
        }
    ]
    response = client.chat.completions.create(
        model="gpt-3.5-turbo-0125",
        messages=messages,
        tools=tools,
        tool_choice="auto",  # auto is default, but we'll be explicit
    )
    response_message = response.choices[0].message
    tool_calls = response_message.tool_calls
    # Step 2: check if the model wanted to call a function
    if tool_calls:
        # Step 3: call the function
        # Note: the JSON response may not always be valid; be sure to handle errors
        available_functions = {
            "get_movie_data": get_movie_recommendations_by_title,
        }  # only one function in this example, but you can have multiple
        messages.append(response_message)  # extend conversation with assistant's reply
        # Step 4: send the info for each function call and function response to the model
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)
            function_response = function_to_call(
                #titles=function_args.get("titles")                
                **function_args
            )
            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response,
                }
            )  # extend conversation with function response
        if function_name == "get_movie_data":
            messages.append({"role": "system", "content": "use the given result from function call to make your movie recommendation. put the streaming proivders behind each recommendation."})
        second_response = client.chat.completions.create(
            model="gpt-3.5-turbo-0125",
            messages=messages,
        )  # get a new response from the model where it can see the function response
    else:
        second_response = response
    return second_response

In [6]:
system_prompt_description = """    
    You are a assistent for movie recommendations. Your goal is to recommend movies based on the users input or answer questions about specific movies. 
    If the user has no specific question recommend 5 movies. Write a commar-seperated list of 5 movie titles.                                                                             
"""
messages = [{"role": "system", "content": system_prompt_description}]


while True:
    user_input = input("You: ")
    if user_input == "quit":
        break

    messages.append({"role": "user", "content": user_input})

    chat_response = run_conversation(messages)


    messages.append({"role": "assistant", "content": chat_response.choices[0].message.content})
    print("Assistant:", chat_response.choices[0].message.content)

_get_movie_data titles input:  ['Fear and Loathing in Las Vegas', 'The Wolf of Wall Street', 'This Is the End', 'Pineapple Express', 'The Nice Guys']
get_basic_data_from_tmdb_for_titles ['Fear and Loathing in Las Vegas', 'The Wolf of Wall Street', 'This Is the End', 'Pineapple Express', 'The Nice Guys']
get_movie_data result:  Title: Fear and Loathing in Las Vegas, Overview: Wir waren kurz hinter Barstow, inmitten der Wüste, als die Drogen anfingen zu wirken...Nevada, 1971: Begleitet von seinem Anwalt Dr. Gonzo rast Raoul Duke in seinem roten Cabrio gen Las Vegas. Für ein Magazin soll er dort von einem Autorennen in der Wüste berichten. Doch für die beiden steckt hinter dem Ausflug sehr viel mehr als eine bloße Reportage. Für sie bedeutet der Trip nach Las Vegas eine Reise in das wilde Herz der von Vietnamkrieg und diverser Skandale gebeutelten Vereinigten Staaten, bei der sie den amerikanischen Traum bei den Hörnern packen und ihm in die hässliche Fratze starren wollen. Ausgerüstet mi

In [48]:
from openai import OpenAI
import json

def run_conversation_2(messages):
    # Step 1: send the conversation and available functions to the model   
    tools = [
        {
            "type": "function",
            "function": {
                "name": "get_movie_recommendations",
                "description": "get all needed information about a list of movietitles regarding streeming providers",
                "_description_": "get the best current movie recommendations based on movie titles with all needed movie data including streaming provider",
                "parameters": {
                    'type': 'object',
                    'properties': {
                        'titles': {
                            'type': 'array',
                            'items': {
                                'type': 'string'                         
                            },
                            'description': 'list of movie titles from given input text'
                        }                        
                    },
                    "required": ["titles"],
                }
            }
        }
    ]
    response = client.chat.completions.create(
        model="gpt-3.5-turbo-0125",
        messages=messages,
        tools=tools,
        tool_choice="auto",  # auto is default, but we'll be explicit
    )
    response_message = response.choices[0].message
    tool_calls = response_message.tool_calls
    # Step 2: check if the model wanted to call a function
    if tool_calls:
        # Step 3: call the function
        # Note: the JSON response may not always be valid; be sure to handle errors
        available_functions = {
            "get_movie_recommendations": get_movie_recommendations_by_title,
        }  # only one function in this example, but you can have multiple
        messages.append(response_message)  # extend conversation with assistant's reply
        # Step 4: send the info for each function call and function response to the model
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)
            function_response = function_to_call(
                #titles=function_args.get("titles")                
                **function_args
            )
            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response,
                }
            )  # extend conversation with function response
        if function_name == "get_movie_recommendations":
            messages.append({"role": "system", "content": "use the given result from function call to make your movie recommendation. put the streaming proivders behind each recommendation."})
        second_response = client.chat.completions.create(
            model="gpt-3.5-turbo-0125",
            messages=messages,
        )  # get a new response from the model where it can see the function response
    else:
        second_response = response
    return second_response

In [50]:
system_prompt_description = """    
    You are a assistent for movie recommendations. Your goal is to recommend movies based on the users input or answer questions about specific movies. 
    If the user has no specific question recommend 5 movies and use the function to specifie the recommendation. Write a commar-seperated list of 5 movie titles.                                                                              
"""
messages = [{"role": "system", "content": system_prompt_description}]


while True:
    user_input = input("You: ")
    if user_input == "quit":
        break

    messages.append({"role": "user", "content": user_input})

    chat_response = run_conversation_2(messages)


    messages.append({"role": "assistant", "content": chat_response.choices[0].message.content})
    print("Assistant:", chat_response.choices[0].message.content)

_get_movie_data titles input:  ['The Wolf of Wall Street', 'Pineapple Express', 'Fear and Loathing in Las Vegas', 'Trainspotting', 'Baby Driver']
get_recro_movies result:  Title: The Wolf of Wall Street, Overview: New York in den frühen 1990er Jahren. Der aufstrebende Aktienhändler Jordan Belfort gründet mit Anfang 20 die Maklerfirma "Stratton Oakmont", mit der er schnell zum Multimillionär aufsteigt und zum Shootingstar der New Yorker Börse wird. Schon bald ist er hauptsächlich unter seinem neuen Spitznamen "Wolf of Wall Street" bekannt. Mit seinem Reichtum finanziert er einen ausschweifenden Lebensstil, der von Alkohol, Drogen, Sex und Dekadenz geprägt ist. Jordans Siegeszug scheint nicht zu stoppen zu sein, doch dies lässt ihn übermütig werden. Von unstillbarer Gier getrieben und mit dem Gefühl der Unbesiegbarkeit im Rücken lassen er und seine "Wolfsbande", darunter sein Kumpel Donnie Azoff, sich auf illegale Geschäfte ein. Das zieht schon bald die Aufmerksamkeit der Justiz auf sich

In [49]:
user_input = "ich möchte lustioge filme mit drogen und musik sehen"

system_prompt_description = """    
    You are a assistent for movie recommendations. Your goal is to recommend movies based on the users input or answer questions about specific movies. 
    If the user has no specific question recommend 5 movies and use the function to specifie the recommendation. Write a commar-seperated list of 5 movie titles.                                                                             
"""
messages = [{"role": "system", "content": system_prompt_description}]


messages.append({"role": "user", "content": user_input})

chat_response = run_conversation_2(messages)


messages.append({"role": "assistant", "content": chat_response.choices[0].message.content})
print("Assistant:", chat_response.choices[0].message.content)

_get_movie_data titles input:  ['Pineapple Express', 'The Wolf of Wall Street', 'Blow', 'Trainspotting', 'Get Him to the Greek']
get_recro_movies result:  Title: The Wolf of Wall Street, Overview: New York in den frühen 1990er Jahren. Der aufstrebende Aktienhändler Jordan Belfort gründet mit Anfang 20 die Maklerfirma "Stratton Oakmont", mit der er schnell zum Multimillionär aufsteigt und zum Shootingstar der New Yorker Börse wird. Schon bald ist er hauptsächlich unter seinem neuen Spitznamen "Wolf of Wall Street" bekannt. Mit seinem Reichtum finanziert er einen ausschweifenden Lebensstil, der von Alkohol, Drogen, Sex und Dekadenz geprägt ist. Jordans Siegeszug scheint nicht zu stoppen zu sein, doch dies lässt ihn übermütig werden. Von unstillbarer Gier getrieben und mit dem Gefühl der Unbesiegbarkeit im Rücken lassen er und seine "Wolfsbande", darunter sein Kumpel Donnie Azoff, sich auf illegale Geschäfte ein. Das zieht schon bald die Aufmerksamkeit der Justiz auf sich, besonders der F